In [ ]:
# File: fishingknot.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import os
import tempfile

import gdown
from frontend import App

# download the fishing knot mesh if not already present
fishingknot_path = os.path.join(tempfile.gettempdir(), "fishingknot.ply")
if not os.path.exists(fishingknot_path):
    fishingknot_id = "1alCDncsmGAFliL1YsPXzqr3pJJvs48P-"
    gdown.download(id=fishingknot_id, output=fishingknot_path)

# create an app
app = App.create("fishingknot")

# load the fishing knot ribbon mesh
V, F = app.mesh.load_tri(fishingknot_path)
app.asset.add.tri("ribbon", V, F)

# create a torus mesh
V, F = app.mesh.torus(r=0.5, R=0.125)
app.asset.add.tri("torus", V, F)

# create a scene
scene = app.scene.create()

# add ribbon with directional color and pin configuration
ribbon = scene.add("ribbon").jitter().direction_color(-1, 0, 0)
ribbon.pin(ribbon.grab([-1, 0, 0], 0.4)).pull(1000.0).move_by([-10, 0, 0], 0, 18)
ribbon.pin(ribbon.grab([0, -1, 0], 0.1))

# set ribbon material properties
(
    ribbon.param.set("model", "arap")
    .set("bend", 100.0)
    .set("young-mod", 1000)
    .set("contact-gap", 1e-3)
    .set("strain-limit", 0.05)
    .set("friction", 0.0)
)

# add torus as a fixed obstacle
torus = scene.add("torus").at(4.55, 1.2, 0.05)
torus.pin()
torus.param.set("friction", 0.0)

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters - disable gravity for this simulation
if app.ci:
    session.param.set("frames", 400)
else:
    session.param.set("frames", 830)
session.param.set("gravity", 0.0)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()